### 1. Carrega o arquivo com as sentenças (Córpus da Gabriela)

In [1]:
import pandas as pd

In [11]:
df = pd.read_csv('tamanho200_3.csv')

In [12]:
df.head()

,adj_polaridade_positiva,pont_repetida,entidade_nomeada,disparadores,expressoes,letras_repetidas,palavras_repetidas,aspas,intensificadores,modificadores,liwc,rotulo,sentenca
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,Parece estar no Reino da Dinamarca.
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,"Esta conversa pra jornalista sabonete ouvir, é..."
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,"Que coisa mais corrupta, dentro de um novo pac..."
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,Dilma elogiando o Congresso porque votaram?
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,pedindo pazes com Temer tipo jardim da infância?


In [13]:
df = df.drop(columns=["pont_repetida","entidade_nomeada", "disparadores", "expressoes", "adj_polaridade_positiva", "letras_repetidas", "aspas", "palavras_repetidas", "liwc", "intensificadores", "modificadores"])

In [17]:
df.head()

,rotulo,sentenca,sentenca_processada1
0,1.0,Parece estar no Reino da Dinamarca.,Parece estar no Reino da Dinamarca.
1,1.0,"Esta conversa pra jornalista sabonete ouvir, é...","Esta conversa pra jornalista sabonete ouvir, é..."
2,1.0,"Que coisa mais corrupta, dentro de um novo pac...","Que coisa mais corrupta, dentro de um novo pac..."
3,1.0,Dilma elogiando o Congresso porque votaram?,Dilma elogiando o Congresso porque votaram?
4,1.0,pedindo pazes com Temer tipo jardim da infância?,pedindo pazes com Temer tipo jardim da infância?


In [6]:
print(df['rotulo'].value_counts())

0.0    200
1.0    200
Name: rotulo, dtype: int64


#### 1.1. Atualiza o DataFrame com apenas as classificações irônicas e não irônicas (2.000 para cada)

In [16]:
sentencas_processadas1 = list()

for sentenca in df['sentenca']:
    sentencas_processadas1.append(sentenca)
    
df['sentenca_processada1'] = sentencas_processadas1


#### 1.2. Realiza um pre-processamento no dataset para remoção de caracteres especiais, vírgulas, pontos etc.

In [18]:
import re

sents = list()
for sentenca in df['sentenca_processada1']:
    sents.append(re.sub(r'[^\w\s]','',sentenca))
    
df['sentenca_processada2'] = sents

### 2. Realiza a obtenção das Features (utiliza sentenca_processada 1 e 2):

In [19]:
df[['sentenca_processada1', 'sentenca_processada2']].head()

,sentenca_processada1,sentenca_processada2
0,Parece estar no Reino da Dinamarca.,Parece estar no Reino da Dinamarca
1,"Esta conversa pra jornalista sabonete ouvir, é...",Esta conversa pra jornalista sabonete ouvir é ...
2,"Que coisa mais corrupta, dentro de um novo pac...",Que coisa mais corrupta dentro de um novo paco...
3,Dilma elogiando o Congresso porque votaram?,Dilma elogiando o Congresso porque votaram
4,pedindo pazes com Temer tipo jardim da infância?,pedindo pazes com Temer tipo jardim da infância


#### 2.1. Entidade Nomeadas

In [21]:
import polyglot
from polyglot.text import Text

naoironicas = 0
ironicas = 0


entidades_nomeadas = list()

count = 0
for sentenca in df['sentenca_processada2']:
    text = Text(sentenca, hint_language_code='pt',)
    if len(text.entities) > 0:
        if df['rotulo'][count] == 1:
            ironicas = ironicas + 1
        if df['rotulo'][count] == 0:
            naoironicas = naoironicas + 1       
        entidades_nomeadas.append(1)
    else:
        entidades_nomeadas.append(0)
        
    count = count + 1
        
            
print(ironicas)
print(naoironicas)

df['entidades_nomeadas'] = entidades_nomeadas

69
62


In [22]:
df['entidades_nomeadas'].value_counts()

0    269
1    131
Name: entidades_nomeadas, dtype: int64

#### 2.2. Adjetivos com polaridade positiva (no domínio político)

In [23]:
import nlpnet
import nltk
import unidecode

stemmer = nltk.RSLPStemmer()

# Define o etiquetador morfosintático do NLPNet:
tagger_nplnet = nlpnet.POSTagger(data_dir='../pos-pt/', language='pt')

# Lista de adjetivos de polaridade positiva bem marcados (Trabalho de Gabriela):
lista_adjetivos_positivos = ['belo', 'bom', 'bem', 'excelente', 'extraordinário', 'fantástico', 'feliz', 'glorioso', 'heróico', 'hilário', 
 'honesto', 'ilustre', 'incrível', 'legal', 'limpo', 'lindo', 'majestoso', 'maravilhoso', 'nobre', 'ótimo', 'santo', 'satisfeito', 
 'solícito']

# recupera as raizes das palavras:
_lista_adjetivos_positivos = [stemmer.stem(palavra) for palavra in lista_adjetivos_positivos]

__lista_adjetivos_positivos = [unidecode.unidecode(palavra) for palavra in _lista_adjetivos_positivos]
  
def contem_adjetivo_positivo_politico(sentenca):
    lista = tagger_nplnet.tag(sentenca)
    for item in lista:
        for (token, tag) in item:
            if tag == 'ADJ' and any(unidecode.unidecode(stemmer.stem(token.lower())) in s for s in __lista_adjetivos_positivos):
                return True
    return False

naoironicas = 0
ironicas = 0

adjetivos = list()

count = 0
for sentenca in df['sentenca_processada2']:
    if contem_adjetivo_positivo_politico(sentenca):
        adjetivos.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        adjetivos.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['adjetivos'] = adjetivos


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


19
4


In [24]:
df['adjetivos'].value_counts()

0    377
1     23
Name: adjetivos, dtype: int64

#### 2.3. Triggers

In [25]:
# Lista de disparadores (Trabalho de Gabriela):
lista_disparadores = ['aceitar', 'acreditar', 'adorar', 'alíquota', 'alma', 'amá-la', 'anjos', 'boa sorte', 'boa viagem', 'boa noite',
  'bocadas', 'bondade', 'campanha', 'canonizá-lo', 'canonização', 'coincidência', 'coisa', 'comprar', 'contribuintes', 'contrição', 
  'corrupção', 'corruptos', 'cortar gastos', 'crescimento', 'crise', 'culpa', 'demitir', 'democracia', 'desemprego', 'desenvolvimento', 
  'desgoverno', 'desgraça', 'destruído', 'desvalorização', 'detalhe', 'deuses', 'dinheirama', 'dinheiro', 'ditador', 'dó', 'eleitoreiras', 
  'embuste', 'estrago', 'ética', 'expertise', 'financiar', 'fome', 'funcionar', 'ganhar', 'gastar', 'gostar', 'gostaria', 'grato', 'honestidade', 
  'ignorância', 'impeachment', 'inflação', 'investimento', 'lambuzou', 'liderança', 'louvor', 'megaempreiteiro', 'mensalão', 'mentiras', 'mídia',
  'milagre', 'miséria', 'moral', 'moralidade', 'necessidade', 'obrigado', 'oposição', 'país', 'parabéns', 'parceria', 'partido', 'pedaladas', 'pena',
  'política', 'político', 'prisão', 'quadrilha', 'quebrou', 'reeileição', 'refúgio', 'rombo', 'salve', 'santo', 'saudade', 'sindicalismo',
  'solidariedade', 'subordinados', 'sugestivo', 'sugestões', 'surpresa', 'tomara', 'tucanas', 'verde-amarelo', 'vermelhos', 'vida']

# recupera as raizes das palavras:
_lista_disparadores = [stemmer.stem(palavra) for palavra in lista_disparadores]

__lista_disparadores = [unidecode.unidecode(palavra) for palavra in _lista_disparadores]
  
def has_trigger(sentenca):
    tokens = nltk.word_tokenize(sentenca)
    for token in tokens:
        if any(unidecode.unidecode(stemmer.stem(token.lower())) in s for s in __lista_disparadores):
            return True
    return False


naoironicas = 0
ironicas = 0

triggers = list()

count = 0
for sentenca in df['sentenca_processada2']:
    if has_trigger(sentenca):
        triggers.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        triggers.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['triggers'] = triggers

187
182


In [26]:
df['triggers'].value_counts()

1    369
0     31
Name: triggers, dtype: int64

#### 2.4. Intensificadores

In [27]:
import nlpnet

# Define o etiquetador morfosintático do NLPNet:
tagger_nplnet = nlpnet.POSTagger(data_dir='../pos-pt/', language='pt')

# Intensificadores
def contem_intensificadores(sentenca):
    lista = tagger_nplnet.tag(sentenca)
    ADV = False
    for item in lista:
         for (_, tag) in item:
            if tag == 'ADV':   
                ADV = True
            elif tag == 'ADJ':
                if(ADV == True):           
                    return True
            else:
                ADV = False
    return False

naoironicas = 0
ironicas = 0

intensifiers = list()

count = 0
for sentenca in df['sentenca_processada2']:
    if contem_intensificadores(sentenca):
        intensifiers.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        intensifiers.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['intensifiers'] = intensifiers

10
10


In [28]:
df['intensifiers'].value_counts()

0    380
1     20
Name: intensifiers, dtype: int64

#### 2.5. Modificadores

In [29]:
lista_modificadores = ['absolutamente', 'afinal', 'agora', 'apesar', 'até', 'aumento', 'como sempre', 'demais', 'do brasil', 'em paz', 'em solidariedade',
 'extremamente', 'ficar longe', 'imagine', 'incondicionalmente', 'lindamente', 'logo agora', 'mais', 'manutenção', 'mas', 'menos',
  'não é verdade', 'obrigatoriamente', 'ou melhor', 'pelo menos', 'pensando bem', 'porque', 'quanta', 'quão', 'sempre tão', 'simplesmente', 'só', 'todos']

_lista_modificadores = [stemmer.stem(palavra) for palavra in lista_modificadores]

__lista_modificadores = [unidecode.unidecode(palavra) for palavra in _lista_modificadores]

def contem_modificadores(sentenca):
    tokens = nltk.word_tokenize(sentenca)
    for token in tokens:
        if any(unidecode.unidecode(stemmer.stem(token.lower())) in s for s in __lista_modificadores):
            return True
    return False

naoironicas = 0
ironicas = 0

modifiers = list()

count = 0
for sentenca in df['sentenca_processada2']:
    if contem_modificadores(sentenca):
        modifiers.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        modifiers.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['modifiers'] = modifiers


186
179


In [30]:
df['modifiers'].value_counts()

1    365
0     35
Name: modifiers, dtype: int64

#### 2.6. Quotation Marks

In [31]:
def contem_palavras_com_aspas(sentenca):
    result = re.findall(r'[\'\"][\w!?\s]+[\'\"]*', sentenca)    
    if len(result) > 0:
        return True
    return False

naoironicas = 0
ironicas = 0

quotation_marks = list()

count = 0
for sentenca in df['sentenca_processada1']:
    if contem_palavras_com_aspas(sentenca):
        quotation_marks.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        quotation_marks.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['quotation_marks'] = quotation_marks

37
6


In [32]:
df['quotation_marks'].value_counts()

0    357
1     43
Name: quotation_marks, dtype: int64

#### 2.7. Pontuação Repetida

In [33]:
def has_repeated_punctuation(sentenca):
    result = re.findall(r'[!?.]+', sentenca)
    for group in result:
        if len(group) >= 2:
            return True        
    return False

naoironicas = 0
ironicas = 0

repeated_punctuation = list()

count = 0
for sentenca in df['sentenca_processada1']:
    if has_repeated_punctuation(sentenca):
        repeated_punctuation.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        repeated_punctuation.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['repeated_punctuation'] = repeated_punctuation

46
24


In [34]:
df['repeated_punctuation'].value_counts()

0    330
1     70
Name: repeated_punctuation, dtype: int64

#### 2.8. Letras Repetidas

In [35]:
def has_letters_repetition(sentenca):
    lista = tagger_nplnet.tag(sentenca)
    for item in lista:
         for (token, tag) in item:
            if(tag == 'N'):
                result = re.findall(r'(\w)\1+', token.lower())
                for group in result:
                    if len(group) >= 1:
                        return True        
    return False

naoironicas = 0
ironicas = 0

letters_repetition = list()

count = 0
for sentenca in df['sentenca_processada1']:
    if has_letters_repetition(sentenca):
        letters_repetition.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        letters_repetition.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['letters_repetition'] = letters_repetition

33
29


In [36]:
df['letters_repetition'].value_counts()

0    338
1     62
Name: letters_repetition, dtype: int64

#### 2.9. Palavras Repetidas

In [37]:
from itertools import groupby

def has_word_repetition(sentenca):
    
    _tokens = nltk.word_tokenize(sentenca.lower())
    
    _raizes = [stemmer.stem(palavra) for palavra in _tokens]

    _sem_acentos = [unidecode.unidecode(palavra) for palavra in _raizes]

    
    if(max([len(list(group)) for key, group in groupby(_sem_acentos)])) > 1:
        return True
    return False

naoironicas = 0
ironicas = 0

word_repetition = list()

count = 0
for sentenca in df['sentenca_processada1']:
    if has_word_repetition(sentenca):
        word_repetition.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        word_repetition.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['word_repetition'] = word_repetition

21
11


In [38]:
df['word_repetition'].value_counts()

0    368
1     32
Name: word_repetition, dtype: int64

#### 2.10. Rethorical Question

In [35]:
# DESENVOLVER!

#### 2.11. LIWC (Executar)

In [39]:
###### LIWC ######
import csv
liwc_words = []

with open('liwc2.txt','r') as f:
    next(f) # skip headings
    reader=csv.reader(f,delimiter='\t')
    for line in reader:
        liwc_words.append(line)

def get_polaridade_palavra(word):
    for line in liwc_words:
        if stemmer.stem(word) in line:
            if '126' in line:
                return 'positivo'
            elif '127' in line:
                return 'negativo'
            else:
                return 'undefined'

    return 'not_found'

def get_polaridade_sentenca(sentenca):
    tokens = nltk.word_tokenize(sentenca.lower())
    positivo = 0
    negativo = 0
    for token in tokens:
        if get_polaridade_palavra(token) == 'positivo':
            positivo = positivo + 1
        elif get_polaridade_palavra(token) == 'negativo':
            negativo = negativo + 1
            

    if (positivo > negativo):
        return 1
    else:
        return 0
    
    
naoironicas = 0
ironicas = 0

liwc = list()

count = 0
for sentenca in df['sentenca_processada1']:
    if get_polaridade_sentenca(sentenca) == 1:
        liwc.append(1)
        if df['rotulo'][count] == 1: ironicas = ironicas + 1
        if df['rotulo'][count] == 0: naoironicas = naoironicas + 1
    else:
        liwc.append(0)
    count = count + 1
    
print(ironicas)
print(naoironicas)

df['liwc'] = liwc

KeyboardInterrupt: 

In [ ]:
df['liwc'].value_counts()

## SALVA DATAFRAME EM CSV:

In [41]:
# Salva o dataframe completo (sentenças + classificação + features)
df.to_csv('dataset_full_400.csv', index=False)

# # remove as colunas das sentenças, permanecendo apenas a classificação e features:
# df1 = df.drop(columns=["index", "sentenca_original", "sentenca_processada1", "sentenca_processada2"])

# # Salva o dataframe features (classificação + features)
# df1.to_csv('dataset_features.csv', index=False)

In [40]:
df.head()

,rotulo,sentenca,sentenca_processada1,sentenca_processada2,entidades_nomeadas,adjetivos,triggers,intensifiers,modifiers,quotation_marks,repeated_punctuation,letters_repetition,word_repetition
0,1.0,Parece estar no Reino da Dinamarca.,Parece estar no Reino da Dinamarca.,Parece estar no Reino da Dinamarca,1,0,1,0,1,0,0,0,0
1,1.0,"Esta conversa pra jornalista sabonete ouvir, é...","Esta conversa pra jornalista sabonete ouvir, é...",Esta conversa pra jornalista sabonete ouvir é ...,0,0,1,0,1,0,0,0,0
2,1.0,"Que coisa mais corrupta, dentro de um novo pac...","Que coisa mais corrupta, dentro de um novo pac...",Que coisa mais corrupta dentro de um novo paco...,0,0,1,1,1,0,0,0,0
3,1.0,Dilma elogiando o Congresso porque votaram?,Dilma elogiando o Congresso porque votaram?,Dilma elogiando o Congresso porque votaram,1,0,1,0,1,0,0,0,0
4,1.0,pedindo pazes com Temer tipo jardim da infância?,pedindo pazes com Temer tipo jardim da infância?,pedindo pazes com Temer tipo jardim da infância,1,0,1,0,1,0,0,0,0


In [45]:
df1.head()

,classificacao,entidades_nomeadas,adjetivos,triggers,intensifiers,modifiers,quotation_marks,repeated_punctuation,letters_repetition,word_repetition
0,1,0,1,1,1,1,0,0,0,0
1,1,0,0,1,0,1,0,1,0,0
2,1,0,0,1,0,1,0,0,1,0
3,1,1,0,1,0,1,1,1,0,0
4,1,0,0,1,0,0,0,1,0,0


In [46]:
df1 = pd.read_csv('dataset_features.csv')

inputs = df1.iloc[:,1:10].values
outputs = df1['classificacao']

### SELEÇÃO DAS FEATURES

In [19]:
import pandas as pd

# Feature Importance
from sklearn import datasets
from sklearn import metrics
# from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree

# load the iris datasets
dataset = pd.read_csv('dataset_features.csv')

inputs = dataset.iloc[:,1:10].values
outputs = dataset['classificacao']

# fit an Extra Trees model to the data
# model = ExtraTreesClassifier()
# model.fit(inputs, outputs)
classificador = tree.DecisionTreeClassifier()
classificador.fit(inputs, outputs)

# display the relative importance of each attribute
print(dataset.iloc[:,1:10].head(0))
print(classificador.feature_importances_)

Empty DataFrame
Columns: [entidades_nomeadas, adjetivos, triggers, intensifiers, modifiers, quotation_marks, repeated_punctuation, letters_repetition, word_repetition]
Index: []
[0.05798318 0.09833265 0.04484432 0.06661862 0.07870014 0.07961864
 0.36655478 0.09197765 0.11537002]


In [36]:
import pandas as pd

# Feature Importance
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# load the iris datasets
dataset = pd.read_csv('dataset_features.csv')

inputs = dataset.iloc[:,1:10].values
outputs = dataset['classificacao']

classificador = LogisticRegression()
# classificador.fit(treino, classe_treino)

# create the RFE model and select 3 attributes
rfe = RFE(classificador, 1)
rfe = rfe.fit(inputs, outputs)

# summarize the selection of the attributes
print(dataset.iloc[:,1:10].head(0))
print(rfe.support_)
print(rfe.ranking_)

Empty DataFrame
Columns: [entidades_nomeadas, adjetivos, triggers, intensifiers, modifiers, quotation_marks, repeated_punctuation, letters_repetition, word_repetition]
Index: []
[False False False False False False  True False False]
[7 2 8 6 4 3 1 5 9]


In [39]:
import pandas as pd

# Feature Importance
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn import svm

# load the iris datasets
dataset = pd.read_csv('dataset_features.csv')

inputs = dataset.iloc[:,1:10].values
outputs = dataset['classificacao']

classificador = svm.SVC(gamma='auto', C=1.0, kernel='linear', probability=False)

# create the RFE model and select 3 attributes
rfe = RFE(classificador, 1)
rfe = rfe.fit(inputs, outputs)

# summarize the selection of the attributes
print(dataset.iloc[:,1:10].head(0))
print(rfe.support_)
print(rfe.ranking_)

Empty DataFrame
Columns: [entidades_nomeadas, adjetivos, triggers, intensifiers, modifiers, quotation_marks, repeated_punctuation, letters_repetition, word_repetition]
Index: []
[False False False False False False  True False False]
[9 2 5 7 3 4 1 8 6]


In [41]:
import pandas as pd

# Feature Importance
from sklearn import datasets
from sklearn import metrics
from sklearn.neural_network import MLPClassifier

# load the iris datasets
dataset = pd.read_csv('dataset_features.csv')

inputs = dataset.iloc[:,1:10].values
outputs = dataset['classificacao']

# fit an Extra Trees model to the data
# model = ExtraTreesClassifier()
# model.fit(inputs, outputs)
classificador = MLPClassifier(activation='relu', solver='adam', max_iter=10000, alpha=1e-10, hidden_layer_sizes=(18,2))
classificador.fit(inputs, outputs)

# display the relative importance of each attribute
print(dataset.iloc[:,1:10].head(0))
print(classificador.feature_importances_)

Empty DataFrame
Columns: [entidades_nomeadas, adjetivos, triggers, intensifiers, modifiers, quotation_marks, repeated_punctuation, letters_repetition, word_repetition]
Index: []


AttributeError: 'MLPClassifier' object has no attribute 'feature_importances_'

In [26]:
# # http://minerandodados.com.br/index.php/2018/05/21/feature-selection-bala-de-prata/
# from sklearn import feature_selection
# fs = feature_selection.SelectPercentile(feature_selection.f_classif, percentile=100)
# # model = feature_selection.SelectKBest(score_func=feature_selection.f_regression, k=9)
# X_treino_fs = fs.fit_transform(inputs, outputs)

# # results = model.fit(df[columns], df['qsec'])

# print(fs.scores_)
# print(fs.pvalues_)

# # x = pd.DataFrame(X_treino_fs)
# # x

# df1.head()